<a href="https://colab.research.google.com/github/oshbocker/CAFA/blob/main/CAFA_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"oshbocker","key":"bb9c6a60ba5c39e689d8cf8d15cb8bca"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install obonet -q
!pip install biopython -q
!pip install torchmetrics -q
!pip install faiss-gpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 21.7 MB/s eta 0:00:00


In [4]:
import os
import torch

from pathlib import Path


iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

cafa_main_path = Path("cafa-5-protein-function-prediction")
cafa_clean_fasta_path = Path("viktorfairuschin/cafa-5-fasta-files")
esm2_path = Path("viktorfairuschin/cafa-5-ems-2-embeddings-numpy")
protbert_path = Path("henriupton/protbert-embeddings-for-cafa5")
t5_path = Path("sergeifironov/t5embeds")

data_paths = [cafa_clean_fasta_path, esm2_path, protbert_path, t5_path]

if iskaggle:
  content_dir = "/kaggle/input"
else:
  content_dir = "/content"

class CFG:
    train_go_obo_path: str = f"{content_dir}/cafa-5-protein-function-prediction/Train/go-basic.obo"
    train_seq_fasta_path: str = f"{content_dir}/cafa-5-protein-function-prediction/Train/train_sequences.fasta"
    train_terms_path: str = f"{content_dir}/cafa-5-protein-function-prediction/Train/train_terms.tsv"
    train_taxonomy_path: str = f"{content_dir}/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv"
    train_ia_path: str = f"{content_dir}/cafa-5-protein-function-prediction/IA.txt"
    test_sequences_path: str = f"{content_dir}/Test (Targets)/testsuperset.fasta"

    num_labels = 500
    n_epochs = 14
    batch_size = 128
    lr = 0.0005

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
if not iskaggle:
  import zipfile,kaggle
  if not cafa_main_path.exists():
    print(cafa_main_path)
    kaggle.api.competition_download_cli(str(cafa_main_path))
    zipfile.ZipFile(f'{cafa_main_path}.zip').extractall(cafa_main_path)
  for data_path in data_paths:
    if not data_path.exists():
      print(data_path)
      kaggle.api.dataset_download_files(str(data_path))
      zipfile.ZipFile(f'{data_path.name}.zip').extractall(data_path.name)

cafa-5-protein-function-prediction


100%|██████████| 115M/115M [00:07<00:00, 17.0MB/s]



viktorfairuschin/cafa-5-fasta-files
viktorfairuschin/cafa-5-ems-2-embeddings-numpy
henriupton/protbert-embeddings-for-cafa5
sergeifironov/t5embeds


In [6]:
import obonet
import time

import networkx as nx
import pandas as pd
import numpy as np
from Bio import SeqIO
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# TORCH MODULES FOR METRICS COMPUTATION :
import torch
from torch.utils.data import Dataset
from torch import nn
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MultilabelF1Score
from torchmetrics.classification import MultilabelAccuracy

In [7]:
# graph = obonet.read_obo(CFG.train_go_obo_path)

In [8]:
# # Information Accretion values for terms
# ia_dict = {}
# with open(CFG.train_ia_path) as f:
#     for line in f:
#         (key, value) = line.split("\t")
#         ia_dict[key] = float(value.replace("\n",""))

In [9]:
# subontology_roots = {'BPO':'GO:0008150',
#                      'CCO':'GO:0005575',
#                      'MFO':'GO:0003674'}

# BPO_nodes = nx.ancestors(graph, subontology_roots['BPO'])
# CCO_nodes = nx.ancestors(graph, subontology_roots['CCO'])
# MFO_nodes = nx.ancestors(graph, subontology_roots['MFO'])

# BPO_len = len(BPO_nodes)
# CCO_len = len(CCO_nodes)
# MFO_len = len(MFO_nodes)
# print(BPO_len, CCO_len, MFO_len, BPO_len + CCO_len + MFO_len)

In [10]:
train_terms = pd.read_csv(CFG.train_terms_path, sep="\t")
# train_terms['ia'] = train_terms['term'].apply(lambda x: ia_dict[x])
print(train_terms.shape)
train_terms.head(5)

(5363863, 3)


,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


## Use embeddings to find similar proteins for classification

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

import faiss

In [12]:
# Directories for the different embedding vectors :
embeds_map = {
    "T5" : "t5embeds",
    "ProtBERT" : "protbert-embeddings-for-cafa5",
    "ESM2" : "cafa-5-ems-2-embeddings-numpy"
}

# Length of the different embedding vectors :
embeds_dim = {
    "T5" : 1024,
    "ProtBERT" : 1024,
    "ESM2" : 1280,
    "Concat": 1280+1024,
}

datatype = "train"
# bert_embeds = np.load(f"{content_dir}/"+embeds_map["ProtBERT"]+"/"+datatype+"_embeddings.npy")
# bert_ids = np.load(f"{content_dir}/"+embeds_map["ProtBERT"]+"/"+datatype+"_ids.npy")
# esm2_embeds = np.load(f"{content_dir}/"+embeds_map["ESM2"]+"/"+datatype+"_embeddings.npy")
# esm2_ids = np.load(f"{content_dir}/"+embeds_map["ESM2"]+"/"+datatype+"_ids.npy")
t5_train_embeds = np.load(f"{content_dir}/"+embeds_map["T5"]+"/"+datatype+"_embeds.npy")
t5_train_embeds = normalize(t5_train_embeds, axis=1, norm='l2').astype(np.float32)
t5_train_ids = np.load(f"{content_dir}/"+embeds_map["T5"]+"/"+datatype+"_ids.npy")

datetype = "test"
t5_test_embeds = np.load(f"{content_dir}/"+embeds_map["T5"]+"/"+datatype+"_embeds.npy")
t5_test_embeds = normalize(t5_test_embeds, axis=1, norm='l2').astype(np.float32)
t5_test_ids = np.load(f"{content_dir}/"+embeds_map["T5"]+"/"+datatype+"_ids.npy")

print(t5_train_embeds.shape)

(142246, 1024)


In [13]:
T5_embed_dict = dict(zip(t5_train_ids, t5_train_embeds))

In [14]:
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse

def get_agg_labels(train_terms, aspect):
  df = train_terms[train_terms['aspect'] == aspect].sort_values('EntryID').reset_index(drop=True)
  idx_dict = df.reset_index().groupby(['EntryID'])['index'].agg(['min', 'max']).apply(lambda rw: (rw['min'], rw['max']+1), axis=1).to_dict()

  ohe = OneHotEncoder()
  aspect_ohe = ohe.fit_transform(train_terms[train_terms['aspect'] == aspect]['term'].values.reshape(-1,1)).toarray()
  agg_labels = np.vstack([aspect_ohe[l:u].sum(axis=0) for l,u in idx_dict.values()])

  del aspect_ohe

  agg_labels_sparse = sparse.csr_matrix(agg_labels)

  del agg_labels

  ids = df['EntryID'].unique()

  return ids, agg_labels_sparse, ohe

In [15]:
BPO_ids, BPO_agg_labels, BPO_ohe = get_agg_labels(train_terms, 'BPO')
CCO_ids, CCO_agg_labels, CCO_ohe = get_agg_labels(train_terms, 'CCO')
MFO_ids, MFO_agg_labels, MFO_ohe = get_agg_labels(train_terms, 'MFO')

In [16]:
# Split into train, val, test
BPO_train_ids, BPO_test_ids, BPO_train_labels, BPO_test_labels = train_test_split(BPO_ids, BPO_agg_labels, test_size = 0.2)
print("BPO")
print("Train label", BPO_train_labels.shape)
print("Test label", BPO_test_labels.shape)
CCO_train_ids, CCO_test_ids, CCO_train_labels, CCO_test_labels = train_test_split(CCO_ids, CCO_agg_labels, test_size = 0.2)
print("CCO")
print("Train label", CCO_train_labels.shape)
print("Test label", CCO_test_labels.shape)
MFO_train_ids, MFO_test_ids, MFO_train_labels, MFO_test_labels = train_test_split(MFO_ids, MFO_agg_labels, test_size = 0.2)
print("MFO")
print("Train label", MFO_train_labels.shape)
print("Test label", MFO_test_labels.shape)

BPO
Train label (73768, 21285)
Test label (18442, 21285)
CCO
Train label (74329, 2957)
Test label (18583, 2957)
MFO
Train label (62909, 7224)
Test label (15728, 7224)


In [17]:
def get_T5_preds(T5_embed_dict, train_ids, test_ids, train_labels, k):
  index = faiss.IndexFlatIP(embeds_dim["T5"])

  # make it into a gpu index
  res = faiss.StandardGpuResources()  # use a single GPU
  gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

  gpu_index.add(np.vstack([T5_embed_dict[p] for p in train_ids]))

  D, I = gpu_index.search(np.vstack([T5_embed_dict[p] for p in test_ids]), k=k)

  preds = []
  for i, p in enumerate(test_ids):
    d = D[i]
    i = I[i]
    weight_vec = (-1/np.log(d))/(-1/np.log(d)).sum()
    weight_vec = weight_vec[..., None]
    top_k_labels = np.vstack(train_labels[i])
    weighted_preds = np.multiply(top_k_labels, weight_vec).sum(axis=0)
    preds.append(weighted_preds)

  return preds

In [18]:
BPO_preds = get_T5_preds(T5_embed_dict, BPO_train_ids, BPO_test_ids, BPO_train_labels, 1)
CCO_preds = get_T5_preds(T5_embed_dict, CCO_train_ids, CCO_test_ids, CCO_train_labels, 2)
MFO_preds = get_T5_preds(T5_embed_dict, MFO_train_ids, MFO_test_ids, MFO_train_labels, 3)

<ipython-input-17-85389e8c8b18>:16: RuntimeWarning: divide by zero encountered in true_divide
  weight_vec = (-1/np.log(d))/(-1/np.log(d)).sum()
<ipython-input-17-85389e8c8b18>:16: RuntimeWarning: invalid value encountered in true_divide
  weight_vec = (-1/np.log(d))/(-1/np.log(d)).sum()


In [19]:
def get_score(test_labels, preds):
  f1_score = MultilabelF1Score(num_labels=test_labels.shape[1]).to('cpu')
  preds = torch.from_numpy(np.vstack([p[0].todense() for p in preds])).to('cpu')
  labels = torch.from_numpy(test_labels.todense()).to('cpu')
  score = f1_score(preds, labels)
  return score

In [20]:
print(get_score(BPO_test_labels, BPO_preds))
print(get_score(CCO_test_labels, CCO_preds))
print(get_score(MFO_test_labels, MFO_preds))

tensor(0.1798)
tensor(0.2154)
tensor(0.2465)


In [21]:
# CCO
# 0.2108 k = 2
# BPO
# 0.1791 k = 1
# 0.1683 k = 2
# 0.1510 k = 3
# MFO
# 0.2561 k = 1


## Make Predictions

In [22]:
BPO_test_preds = get_T5_preds(T5_embed_dict, BPO_ids, t5_test_ids, BPO_agg_labels, 1)
CCO_test_preds = get_T5_preds(T5_embed_dict, CCO_ids, t5_test_ids, CCO_agg_labels, 1)
MFO_test_preds = get_T5_preds(T5_embed_dict, MFO_ids, t5_test_ids, MFO_agg_labels, 1)

<ipython-input-17-85389e8c8b18>:16: RuntimeWarning: divide by zero encountered in true_divide
  weight_vec = (-1/np.log(d))/(-1/np.log(d)).sum()
<ipython-input-17-85389e8c8b18>:16: RuntimeWarning: invalid value encountered in true_divide
  weight_vec = (-1/np.log(d))/(-1/np.log(d)).sum()


In [42]:
def create_submission_df(ids, test_preds, ohe):
  id_list = []
  term_list = []
  confidence_list = []

  for idx, protein in enumerate(ids):
    protein_preds = test_preds[idx][0]
    pred_idx = protein_preds.indices
    pred_terms = [ohe.categories_[0][i] for i in pred_idx]
    pred_values = list(protein_preds.data)
    num_preds = len(pred_values)

    id_list += [protein] * num_preds
    term_list += pred_terms
    confidence_list += pred_values

  submission_df = pd.DataFrame({'Id': id_list, 'GO term': term_list, 'Confidence': confidence_list})

  return submission_df


In [44]:
BPO_df = create_submission_df(BPO_ids, BPO_test_preds, BPO_ohe)
CCO_df = create_submission_df(CCO_ids, CCO_test_preds, CCO_ohe)
MFO_df = create_submission_df(MFO_ids, MFO_test_preds, MFO_ohe)

In [45]:
submission_df = pd.concat([BPO_df, CCO_df, MFO_df])

In [47]:
submission_df.to_csv('submission.tsv', sep='\t', index=False)